In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import cross_val_score ,GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

In [3]:
heart_data = pd.read_csv('heart.csv')
heart_data.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [4]:
heart_data.shape

(303, 14)

In [5]:
heart_data.columns

Index(['age', 'sex', 'cp', 'trtbps', 'chol', 'fbs', 'restecg', 'thalachh',
       'exng', 'oldpeak', 'slp', 'caa', 'thall', 'output'],
      dtype='object')

In [6]:
heart_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trtbps    303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalachh  303 non-null    int64  
 8   exng      303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slp       303 non-null    int64  
 11  caa       303 non-null    int64  
 12  thall     303 non-null    int64  
 13  output    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [7]:
heart_data.isnull().sum()

age         0
sex         0
cp          0
trtbps      0
chol        0
fbs         0
restecg     0
thalachh    0
exng        0
oldpeak     0
slp         0
caa         0
thall       0
output      0
dtype: int64

In [8]:
heart_data.describe()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [9]:
x = heart_data.drop('output',axis=1)
y = heart_data['output']

In [10]:
x,y

(     age  sex  cp  trtbps  chol  fbs  restecg  thalachh  exng  oldpeak  slp  \
 0     63    1   3     145   233    1        0       150     0      2.3    0   
 1     37    1   2     130   250    0        1       187     0      3.5    0   
 2     41    0   1     130   204    0        0       172     0      1.4    2   
 3     56    1   1     120   236    0        1       178     0      0.8    2   
 4     57    0   0     120   354    0        1       163     1      0.6    2   
 ..   ...  ...  ..     ...   ...  ...      ...       ...   ...      ...  ...   
 298   57    0   0     140   241    0        1       123     1      0.2    1   
 299   45    1   3     110   264    0        1       132     0      1.2    1   
 300   68    1   0     144   193    1        1       141     0      3.4    1   
 301   57    1   0     130   131    0        1       115     1      1.2    1   
 302   57    0   1     130   236    0        0       174     0      0.0    1   
 
      caa  thall  
 0      0      1  


In [11]:
heart_data['output'].value_counts()

1    165
0    138
Name: output, dtype: int64

# 1 -> have heart dieases
# 0 ->  healthy heart

In [13]:
X = np.asarray(x)
Y = np.asarray(y)

# model selection

In [29]:
# select the model by default parameters using cross validation
# list the modle
models = [LogisticRegression(max_iter=1000) , SVC(kernel='linear') , RandomForestClassifier(random_state=0) , KNeighborsClassifier()]

In [34]:
def cross_validiation_score_cv():
    for model in models:
        cv = cross_val_score(model,X,Y,cv=5)
        mean_cv = sum(cv) / len(cv)
        rou_cv = round(mean_cv,2)
        print("The cross validation score of ",model ,"is in (%):- ",rou_cv*100,"%")
        print("--------------------------------------------------------------------------")

In [35]:
cross_validiation_score_cv()

C:\Users\khadeer\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


The cross validation score of  LogisticRegression(max_iter=1000) is in (%):-  83.0 %
--------------------------------------------------------------------------
The cross validation score of  SVC(kernel='linear') is in (%):-  83.0 %
--------------------------------------------------------------------------
The cross validation score of  RandomForestClassifier(random_state=0) is in (%):-  84.0 %
--------------------------------------------------------------------------
The cross validation score of  KNeighborsClassifier() is in (%):-  64.0 %
--------------------------------------------------------------------------


# Here Randomforestclassfifer model have highest accuracy score in it....
# 84% percentage 

In [37]:
# now comparing the models with different hyperparameters using gridsearchcv

In [36]:
models_list = [LogisticRegression(max_iter=1000) , SVC(kernel='linear') , RandomForestClassifier(random_state=0) , KNeighborsClassifier()]

In [38]:
# set the parameters by above models
model_parameters = {
        'log_parameters':{
            'C':[1,5,10,20]
        },
        'cv_parameters':{
            'kernal':['linear','ploy','rfb','sigmoid'],
            'C':[1,5,10,20]
        },
        'knn_parameters':{
            'n_neighbors':[3,5,10]
        },
        'random_forest_classifier':{
            'n_estimaters':[10,20,50,100]
        }
}

In [39]:
type(model_parameters)

dict

In [44]:
model_keys = list(model_parameters.keys())
print(model_keys)

['log_parameters', 'cv_parameters', 'knn_parameters', 'random_forest_classifier']


In [66]:
def model_seletion(list_of_models,hyperparameters_dictoniers):
    result = []
    i = 0
    for model in list_of_models:
        keys = model_keys[i]
        params = hyperparameters_dictoniers[keys]
        i += 1
        # print(model)
        # print(keys)

        classifier = GridSearchCV(model,params,cv=5)
        classifier.fit(x,y)
        result.append(
            {
                'model_used':model,
                'highest_score':classifier.best_score_,
                'best_params':classifier.best_params_
            }
        )

    result_df = pd.DataFrame(result,columns=['model_used','highest_score','best_params'])
    return result_df

In [67]:
model_seletion(models_list,model_parameters)

C:\Users\khadeer\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\khadeer\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linea

ValueError: Invalid parameter 'kernal' for estimator SVC(C=1, kernel='linear'). Valid parameters are: ['C', 'break_ties', 'cache_size', 'class_weight', 'coef0', 'decision_function_shape', 'degree', 'gamma', 'kernel', 'max_iter', 'probability', 'random_state', 'shrinking', 'tol', 'verbose'].